<a href="https://colab.research.google.com/github/FaizaAli-Dev/AI-DS-Projects/blob/All-about-AI-and-Data-Science/Checking_Over_and_Underfitting_on_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
# Set random seed for reproducibility
tf.random.set_seed(42)

# For tensorboard
import pathlib
import shutil
import tempfile
import datetime
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Clear any logs from previous runs
#rm -rf ./logs/



In [2]:
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)

In [ ]:
# Load and preprocess the MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

# Define the CNN model
def create_cnn_model():
    model = Sequential()

    # Add convolutional layers with batch normalization
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))

    model.add(BatchNormalization())
   #
    model.add(MaxPooling2D((2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))

    model.add(BatchNormalization())
   #
    model.add(MaxPooling2D((2, 2)))
  #
    model.add(Flatten())

    # Add fully connected layers with dropout
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
    model.add(Dropout(0.5))

    model.add(Dense(10, activation='softmax'))

    return model

# Create an instance of the CNN model
model = create_cnn_model()

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Data augmentation
datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1)

# The patience parameter is set to 3 number of epochs or iterations,
# The training will terminate to 3
# if there is no improvement in the monitor performance measure for X epochs or iterations in a row. It will keep working
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Train the model with data augmentation and early stopping
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    steps_per_epoch=len(X_train) // 32,
                    epochs=2,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping])
# 13 epochs would be better

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

"""
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
"""

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/2
1875/1875 [==============================] - 109s 57ms/step - loss: 0.8903 - accuracy: 0.8416 - val_loss: 0.2966 - val_accuracy: 0.9769
Epoch 2/2
 282/1875 [===>..........................] - ETA: 1:29 - loss: 0.4429 - accuracy: 0.9285

In [ ]:
%tensorboard --logdir logs/fit